In [316]:
import pandas as pd
import gym
import keras
import numpy as np
import collections
from keras.models import Sequential
from keras.layers import Dense
import random
env = gym.make("LunarLander-v2")

In [317]:
gamma = 0.1
alpha = 0.3
seed = 125545
episodes = 100
epsilon = 0.3
np.random.seed(seed)
env.seed(seed)
action_space = env.action_space.n
observation_space = env.observation_space.shape[0]


In [318]:
def get_action(model, state,  epsilon):
        if np.random.random() < epsilon:  
            return np.random.randint(4) 
        else:
            #state = np.transpose(state)
            state = np.array([state])
            result = model.predict(state)
            action_index = np.argmax(result)
            return action_index

In [319]:
def createQNetwork(action_space, observation_space):
    model = Sequential()
    model.add(Dense(12, input_dim=observation_space, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(action_space, activation='sigmoid'))
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    return model

In [332]:
def train():
    state_memory = collections.deque()
    next_state_memory = collections.deque()
    reward_memory = collections.deque()
    done_memory = collections.deque()
    action_memory = collections.deque()
    D = collections.deque()

    model = createQNetwork(action_space, observation_space)
    # target = createQNetwork(action_space, observation_space)
    for _ in range(episodes):
        state = env.reset()
        done = False

        while not done:
            # env.render()
            # take action
            action = get_action(model, state, epsilon)
            # next step
            new_state, reward, done, _ = env.step(action)

            # save memory
            # state_memory.append(state)
            # next_state_memory.append(new_state)
            # reward_memory.append(reward)
            # done_memory.append(done)
            # action_memory.append(action)


            D.append([state, action, reward, new_state, done])
            try:
                sample_D = random.sample(D, 100)
            except ValueError as e:
                sample_D = D

            targets_q = []
            states_to_train_with = []
            # calculate target
            for i in range(0, len(sample_D)):
                output = model.predict(np.array([sample_D[i][3]]))
                is_terminal = sample_D[i][4]
                target_q = model.predict(np.array([sample_D[i][0]]))
                # if terminal, only equals reward
                if is_terminal:
                    target_q[0][sample_D[i][1]] = sample_D[i][2]
                else:
                    target_q[0][sample_D[i][1]] = sample_D[i][2] + gamma * np.max(output)
                    
                targets_q.append(target_q[0])
                states_to_train_with.append(sample_D[i][0])
            
            model.fit(np.array(states_to_train_with), np.array(targets_q), epochs=150, batch_size=10, verbose=0)
            
            
            state = new_state

            # model = tf.keras.models.clone_model(target)

In [333]:
train()

KeyboardInterrupt: 